In [1]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=14c7f7a534b39722db1b16a97c97a0611e92c10bcf2ab7311a95e0112d9fa4f0
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [2]:
import pandas as pd
import openai
import time
from fpdf import FPDF

openai.api_key = ''

In [3]:
url = 'https://raw.githubusercontent.com/cherrymaegrace/rag-logistics-sample/refs/heads/main/datasets/parcel_information_dataset.csv'
dataset = pd.read_csv(url)

dataset.head()

,Parcel ID,Sender Name,Recipient Name,Parcel Weight (kg),Shipment Date,Expected Delivery Date,Current Status,Carrier,Delivery Notes,Insurance Value,Payment Status,Shipping Cost
0,100001,ABC Corp,John Doe,2.5,2024-11-01,2024-11-03,Out for Delivery,FedEx,Leave at front door,150.0,Paid,20.0
1,100002,XYZ Ltd,Jane Smith,1.2,2024-11-02,2024-11-04,In Transit,UPS,Signature required,200.0,COD,15.0
2,100003,Global Ex,Michael Tan,3.0,2024-11-03,2024-11-05,Delivered,DHL,NaN,300.0,Paid,25.0
3,100004,Omega Tech,Lisa Wong,0.8,2024-11-01,2024-11-03,Out for Delivery,USPS,Deliver to reception,100.0,Paid,10.0
4,100005,Delta Inc,Robert King,5.0,2024-11-01,2024-11-05,In Transit,FedEx,"Fragile, handle with care",500.0,Paid,50.0


In [4]:
sample_data = dataset.sample(5)

sample_data

,Parcel ID,Sender Name,Recipient Name,Parcel Weight (kg),Shipment Date,Expected Delivery Date,Current Status,Carrier,Delivery Notes,Insurance Value,Payment Status,Shipping Cost
14,100015,NextGen Shp,Justin Reed,3.5,2024-11-01,2024-11-05,In Transit,USPS,Call upon delivery,350.0,Paid,25.0
2,100003,Global Ex,Michael Tan,3.0,2024-11-03,2024-11-05,Delivered,DHL,NaN,300.0,Paid,25.0
9,100010,JetLog,Sarah Miller,6.5,2024-11-03,2024-11-07,In Transit,FedEx,NaN,600.0,Paid,55.0
16,100017,AceLogistics,Brian Thompson,5.7,2024-11-01,2024-11-06,Awaiting Dispatch,DHL,Fragile,550.0,Paid,45.0
11,100012,Express Co,Hannah White,1.1,2024-11-02,2024-11-04,Out for Delivery,FedEx,NaN,150.0,Paid,12.0


# Context and Form Extaction

Understanding your data

In [9]:
# Get Column Names and Data Types
dataset['Parcel ID'] = dataset['Parcel ID'].astype('object')
columns_info = dataset.dtypes
print(columns_info)

Parcel ID                  object
Sender Name                object
Recipient Name             object
Parcel Weight (kg)        float64
Shipment Date              object
Expected Delivery Date     object
Current Status             object
Carrier                    object
Delivery Notes             object
Insurance Value           float64
Payment Status             object
Shipping Cost             float64
dtype: object


In [10]:
# Identify Categorical and Numerical Features
categorical_features = dataset.select_dtypes(include=['object']).columns
numerical_features = dataset.select_dtypes(include=['int64', 'float64']).columns

print("Categorical Features:", categorical_features)
print("Numerical Features:", numerical_features)

Categorical Features: Index(['Parcel ID', 'Sender Name', 'Recipient Name', 'Shipment Date',
       'Expected Delivery Date', 'Current Status', 'Carrier', 'Delivery Notes',
       'Payment Status'],
      dtype='object')
Numerical Features: Index(['Parcel Weight (kg)', 'Insurance Value', 'Shipping Cost'], dtype='object')


In [11]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(column, limit=10):
    unique_values = dataset[column].unique().tolist()

    if len(unique_values) > limit:
        unique_values = unique_values[:limit] + ['...']
    return unique_values

unique_values = {col: get_unique_values(col) for col in categorical_features}
for col, uniques in unique_values.items():
    print(f"{col}: {uniques}")

Parcel ID: [100001, 100002, 100003, 100004, 100005, 100006, 100007, 100008, 100009, 100010, '...']
Sender Name: ['ABC Corp', 'XYZ Ltd', 'Global Ex', 'Omega Tech', 'Delta Inc', 'FastShip', 'QuickEx', 'Speedy Co', 'Alpha Shp', 'JetLog', '...']
Recipient Name: ['John Doe', 'Jane Smith', 'Michael Tan', 'Lisa Wong', 'Robert King', 'Maria Perez', 'Thomas Lee', 'Emily Davis', 'Kevin Brown', 'Sarah Miller', '...']
Shipment Date: ['2024-11-01', '2024-11-02', '2024-11-03']
Expected Delivery Date: ['2024-11-03', '2024-11-04', '2024-11-05', '2024-11-06', '2024-11-07']
Current Status: ['Out for Delivery', 'In Transit', 'Delivered', 'Awaiting Dispatch']
Carrier: ['FedEx', 'UPS', 'DHL', 'USPS']
Delivery Notes: ['Leave at front door', 'Signature required', nan, 'Deliver to reception', 'Fragile, handle with care', 'Leave at side gate', 'Call upon arrival', 'Fragile', 'Deliver to neighbor', 'Call upon delivery']
Payment Status: ['Paid', 'COD', 'Pending']


In [12]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = dataset[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n", numerical_stats, "\n")

Descriptive Statistics for Numerical Features:
        Parcel Weight (kg)  Insurance Value  Shipping Cost
count           20.000000         20.00000      20.000000
mean             3.040000        291.50000      24.250000
std              1.653831        143.42704      13.013657
min              0.800000        100.00000      10.000000
25%              1.800000        180.00000      14.750000
50%              2.850000        275.00000      21.000000
75%              4.150000        360.00000      28.500000
max              6.500000        600.00000      55.000000 



#Template Generation

In [14]:
def generate_template(dataset, columns_info, categorical_features, numerical_features, unique_values, numerical_stats):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for column, dtype in columns_info.items():
        column_summary += f"{column}: {dtype}\n"

    # Unique values for categorical features
    unique_values_str = "\nUnique Values for Categorical Features:\n"
    for col, uniques in unique_values.items():
        unique_values_str += f"{col}: {uniques}\n"

    # Descriptive statistics for numerical features
    numerical_stats_str = "\nDescriptive Statistics for Numerical Features:\n"
    for col in numerical_features:
        numerical_stats_str += f"{col}: \n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"   {stat_name}: {value}\n"

    # Define the system prompt
    system_prompt = """
    You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness.
    """

    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."

    Output only the template without any explanation or introduction.
    The template's variables will be dynamically replaced so make sure they're formatted properly
    """

    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None

In [15]:
# Generate the template
template = generate_template(dataset, columns_info, categorical_features, numerical_features, unique_values, numerical_stats)
print("Generated Template:", template)


Generated Template: "Parcel ID {Parcel ID} was shipped by {Sender Name} to {Recipient Name} on {Shipment Date}, weighing {Parcel Weight (kg)} kg. The expected delivery date is {Expected Delivery Date}, and the current status of the parcel is '{Current Status}' with the carrier being {Carrier}. Delivery notes indicate: '{Delivery Notes}'. The parcel is insured for a value of ${Insurance Value} and the payment status is '{Payment Status}'. The shipping cost for this parcel was ${Shipping Cost}."


#Template Population

In [16]:
def populate_template(row, template):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict = row.to_dict()
    for key, value in row_dict.items():
      if pd.isna(value):
        row_dict[key] = 'N/A'

    # Replace placeholders in the template with actual values
    populated_template = template.format(**row_dict)
    return populated_template

In [19]:
# Apply the populate_template function to each row in the dataframe
dataset['paragraph'] = dataset.apply(lambda row: populate_template(row, template), axis=1)

dataset[.head()]

,Parcel ID,Sender Name,Recipient Name,Parcel Weight (kg),Shipment Date,Expected Delivery Date,Current Status,Carrier,Delivery Notes,Insurance Value,Payment Status,Shipping Cost,paragraph
0,100001,ABC Corp,John Doe,2.5,2024-11-01,2024-11-03,Out for Delivery,FedEx,Leave at front door,150.0,Paid,20.0,"""Parcel ID 100001 was shipped by ABC Corp to J..."
1,100002,XYZ Ltd,Jane Smith,1.2,2024-11-02,2024-11-04,In Transit,UPS,Signature required,200.0,COD,15.0,"""Parcel ID 100002 was shipped by XYZ Ltd to Ja..."
2,100003,Global Ex,Michael Tan,3.0,2024-11-03,2024-11-05,Delivered,DHL,NaN,300.0,Paid,25.0,"""Parcel ID 100003 was shipped by Global Ex to ..."
3,100004,Omega Tech,Lisa Wong,0.8,2024-11-01,2024-11-03,Out for Delivery,USPS,Deliver to reception,100.0,Paid,10.0,"""Parcel ID 100004 was shipped by Omega Tech to..."
4,100005,Delta Inc,Robert King,5.0,2024-11-01,2024-11-05,In Transit,FedEx,"Fragile, handle with care",500.0,Paid,50.0,"""Parcel ID 100005 was shipped by Delta Inc to ..."


In [20]:
dataset['paragraph'][0]

'"Parcel ID 100001 was shipped by ABC Corp to John Doe on 2024-11-01, weighing 2.5 kg. The expected delivery date is 2024-11-03, and the current status of the parcel is \'Out for Delivery\' with the carrier being FedEx. Delivery notes indicate: \'Leave at front door\'. The parcel is insured for a value of $150.0 and the payment status is \'Paid\'. The shipping cost for this parcel was $20.0."'

#Saving the Populated Templates

In [37]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")

In [38]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [39]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(dataset['paragraph'].to_list(),
                    txt_filename='paragraphs.txt')

save_content_to_pdf(dataset['paragraph'].to_list(),
                    pdf_filename='paragraphs.pdf')

All content has been saved as 'paragraphs.txt'.

All content has been saved as 'paragraphs.pdf'.

